# Python Environment & Packaging — **Interactive Lab**

**Updated:** October 17, 2025

This hands-on notebook guides you through:
- Installing Python (concepts + verifications)
- Virtual environments (`venv`) and version management (`pyenv` overview)
- Dependency management (`pip`, `requirements.txt`, `pyproject.toml`)
- Packaging with `pyproject.toml` (setuptools / hatchling)
- Building distributions (`sdist`, `wheel`) and publishing (TestPyPI with `twine`)
- Workflow & automation (Poetry, Hatch, tox, GitHub Actions)
- Documentation (Sphinx, Read the Docs)

## Table of Contents
- [1. Verify Python Install](#1-verify-python-install)
- [2. Virtual Environments (venv)](#2-virtual-environments-venv)
- [3. Version Management (pyenv overview)](#3-version-management-pyenv-overview)
- [4. Dependency Management](#4-dependency-management)
- [5. Package Structure & `src` Layout](#5-package-structure--src-layout)
- [6. `pyproject.toml` & Build Backends](#6-pyprojecttoml--build-backends)
- [7. Build Distributions (`sdist` & `wheel`)](#7-build-distributions-sdist--wheel)
- [8. Publish to TestPyPI with Twine](#8-publish-to-testpypi-with-twine)
- [9. Workflow & Automation (Poetry, Hatch, tox, CI)](#9-workflow--automation-poetry-hatch-tox-ci)
- [10. Documentation (Sphinx, RTD)](#10-documentation-sphinx-rtd)
- [11. Capstone Lab](#11-capstone-lab)

## 1. Verify Python Install

In [ ]:
import sys, shutil, platform
print("Python:", sys.version.split()[0])
print("Executable:", sys.executable)
print("Platform:", platform.platform())
print("pip path:", shutil.which("pip"))

## 2. Virtual Environments (venv)

Create and activate a virtual environment **locally** (commands for your OS):

**Windows (PowerShell):**
```powershell
python -m venv .venv
.venv\Scripts\Activate.ps1
python -m pip install --upgrade pip
```

**macOS/Linux (bash/zsh):**
```bash
python -m venv .venv
source .venv/bin/activate
python -m pip install --upgrade pip
```

> In this hosted environment, activation isn't demonstrated. The following cells *simulate* checks.

In [ ]:
# Simulate verifying an environment: list installed packages (current kernel env)
import pkgutil
mods = sorted(m.name for m in pkgutil.iter_modules())
print("Sample of available modules:", mods[:20])

**Exercise:** Create a venv locally, install `requests`, then run `python -c "import requests; print(requests.__version__)"`.

## 3. Version Management (pyenv overview)

`pyenv` lets you install/switch Python versions.

Install (macOS with Homebrew):
```bash
brew update && brew install pyenv
pyenv install 3.12.7
pyenv global 3.12.7
pyenv local 3.10.14
```
Check:
```bash
pyenv versions
python --version
```

## 4. Dependency Management

Use `pip` & `requirements.txt`:

```bash
pip install requests flask
pip freeze > requirements.txt
pip install -r requirements.txt
```

Or use **PEP 621** style dependencies in `pyproject.toml` (Poetry example shown later).

In [ ]:
# Quick demo: parsing a requirements-style list (simulate freeze usage)
reqs = ['requests>=2.32.0', 'flask>=3.0.0']
print("\n".join(reqs))

## 5. Package Structure & `src` Layout

Recommended layout (PyPA):

```
myproject/
├─ pyproject.toml
├─ README.md
├─ LICENSE
├─ src/
│  └─ myproject/
│     ├─ __init__.py
│     └─ math_utils.py
└─ tests/
   └─ test_math_utils.py
```

In [ ]:
# Let's generate an example project tree in /mnt/data (no builds executed)
from pathlib import Path
base = Path("/mnt/data/myproject")
(base / "src/myproject").mkdir(parents=True, exist_ok=True)
(base / "tests").mkdir(parents=True, exist_ok=True)

(base / "src/myproject/__init__.py").write_text("__version__ = '0.1.0'\n", encoding="utf-8")
(base / "src/myproject/math_utils.py").write_text("def add(a,b):\n    return a+b\n", encoding="utf-8")
(base / "README.md").write_text("# myproject\nDemo package.\n", encoding="utf-8")
(base / "LICENSE").write_text("MIT\n", encoding="utf-8")

print("Created:", base)

## 6. `pyproject.toml` & Build Backends

**Option A — setuptools** (minimal):

```toml
[build-system]
requires = ["setuptools>=68", "wheel"]
build-backend = "setuptools.build_meta"

[project]
name = "myproject"
version = "0.1.0"
description = "Demo package"
readme = "README.md"
requires-python = ">=3.9"
authors = [{ name = "Your Name" }]
dependencies = []
```

**Option B — hatchling:**
```toml
[build-system]
requires = ["hatchling"]
build-backend = "hatchling.build"

[project]
name = "myproject"
version = "0.1.0"
description = "Demo package"
readme = "README.md"
requires-python = ">=3.9"
authors = [{ name = "Your Name" }]
dependencies = []
```

**Option C — poetry-core (managed by Poetry)** is auto-generated by `poetry init`.

In [ ]:
# Write a minimal setuptools-style pyproject.toml to the demo project
from pathlib import Path
pyproject = Path("/mnt/data/myproject/pyproject.toml")
pyproject.write_text('''[build-system]
requires = ["setuptools>=68", "wheel"]
build-backend = "setuptools.build_meta"

[project]
name = "myproject"
version = "0.1.0"
description = "Demo package"
readme = "README.md"
requires-python = ">=3.9"
authors = [{ name = "Demo Author" }]
dependencies = []
''', encoding="utf-8")
print("Wrote", pyproject)

## 7. Build Distributions (`sdist` & `wheel`)

To build locally (requires `build`):

```bash
python -m pip install build
python -m build
```
This creates `dist/myproject-0.1.0.tar.gz` (sdist) and `dist/myproject-0.1.0-py3-none-any.whl` (wheel).

In [ ]:
# Show where the dist would live
from pathlib import Path
print("Dist path would be:", Path("/mnt/data/myproject/dist"))

## 8. Publish to TestPyPI with Twine

**Safest first step**: publish to **TestPyPI**

```bash
python -m pip install twine
twine upload --repository testpypi dist/*
# Then: pip install -i https://test.pypi.org/simple/ myproject==0.1.0
```
Make sure you have accounts & API tokens set up.

## 9. Workflow & Automation (Poetry, Hatch, tox, CI)

**Poetry** (one-tool flow):
```bash
curl -sSL https://install.python-poetry.org | python -
poetry init
poetry add requests
poetry build
poetry publish --dry-run
```

**Hatch** (project management):
```bash
pip install hatch
hatch new myproject
hatch build
```

**tox** (test automation across versions):
```ini
# tox.ini
[tox]
envlist = py310, py311

[testenv]
deps = pytest
commands = pytest -q
```

**GitHub Actions** (CI):
```yaml
name: ci
on: [push, pull_request]
jobs:
  test:
    runs-on: ubuntu-latest
    strategy:
      matrix:
        python-version: ["3.10", "3.11"]
    steps:
      - uses: actions/checkout@v4
      - uses: actions/setup-python@v5
        with: { python-version: ${{ matrix.python-version }} }
      - run: python -m pip install -U pip build
      - run: pip install -r requirements.txt || true
      - run: pip install pytest
      - run: pytest -q
```

## 10. Documentation (Sphinx, RTD)

Initialize Sphinx:
```bash
python -m pip install sphinx
sphinx-quickstart docs
```
Autodoc minimal example:
```rst
.. automodule:: myproject.math_utils
   :members:
```
Read the Docs config:
```yaml
# .readthedocs.yaml
version: 2
sphinx:
  configuration: docs/conf.py
python:
  install:
    - method: pip
      path: .
```

## 11. Capstone Lab

**Goal:** Build, package, and (optionally) publish a `string_tools` library.

- Implement functions: `camel_to_snake`, `is_palindrome`
- Add tests in `tests/`
- Create `pyproject.toml`, build wheels, and upload to TestPyPI

In [ ]:
# Starter stubs (drop these into your src/ package files locally)
def camel_to_snake(s:str)->str:
    import re
    return re.sub(r'(?<!^)([A-Z])', r'_\1', s).lower()

def is_palindrome(s:str)->bool:
    t = ''.join(ch.lower() for ch in s if ch.isalnum())
    return t == t[::-1]

print(camel_to_snake("CamelCaseToSnake"))
print(is_palindrome("A man, a plan, a canal: Panama"))